In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import os
from folium.plugins import HeatMap

In [ ]:
years = range(2022,2026)
df_all = pd.concat(
    [pd.read_csv(f'../data/raw/feminicide_{year}.csv') for year in years],
    ignore_index=True
)

In [94]:
df_all.describe(include='all')

,QUAND,LIEU,DÉPARTEMENT,PRÉNOM,ÂGE,N°,CONTENU PUBLICATION,LIEN INSTAGRAM
count,529,529,529,528,527.000000,529.000000,528,528
unique,429,466,95,303,NaN,NaN,525,525
top,14/01/2022,Marseille,Paris,–,NaN,NaN,Double féminicide:\n\nEntre les 9 & 14 avril à...,https://www.instagram.com/p/Cmju0uVNGm4/
freq,3,10,23,150,NaN,NaN,2,2
mean,NaN,NaN,NaN,NaN,47.785579,67.631380,NaN,NaN
std,NaN,NaN,NaN,NaN,20.139946,39.781056,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,32.000000,34.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,45.000000,67.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,61.500000,100.000000,NaN,NaN


In [97]:
df_all = df_all.rename(columns=lambda x: x.strip().lower().replace(' ', '_'))

In [105]:
df_all.columns = (
    df_all.columns
    .str.normalize("NFKD")
    .str.encode("ascii", "ignore")
    .str.decode("utf-8")
)

In [106]:

#suppression colonnes inutiles
df_all.head()


,date,commune,departement,prenom,age
0,25/11/2022,Prahecq,Deux-Sèvres,Leslie,22.0
1,04/01/2023,Perles,Aisne,–,50.0
2,31/12/2022,Charleville-Mézières,Ardennes,–,23.0
3,24/12/2022,Saint-Raphaël,Var,Molka,31.0
4,16/10/2022,Goudelin,Côtes-d'Armor,Jacqueline,67.0


In [107]:
df_all['commune'].unique()

array(['Prahecq', 'Perles', 'Charleville-Mézières', 'Saint-Raphaël ',
       'Goudelin', 'Soula', '13e arrondissement', 'Mulhouse', 'Lannoy',
       'Pantin', 'Mont-Saint-Martin', 'Tampon', 'Floirac', 'Grand-Fayt',
       'Villeurbanne', "Saint-Pierre-d'Eyraud", 'Valframbert',
       'Saint-Jean-de-Gonville', 'Rouen', 'Quimper', 'Brecé',
       'Évry-Courcouronnes', 'Pargny-sur-Saulx', 'Pavillons-sous-Bois ',
       'Saint-Jacques-de-la-Lande', 'Saint-Jacques-de-la-Lande ',
       'Louviers ', 'Tonneins', '10e arrondissement', 'Savigné',
       'Montaren-et-Saint-Médiers', 'Diancey', "Berre-l'Etang", 'Ronchin',
       'Ducos', 'Sainte-Rose', 'Stains', 'Neuvy ', 'Carantec', 'Meudon',
       'Ille-sur-Têt', 'Brive la Gaillarde', 'Nantes', 'Ajaccio',
       'Taravao', 'Blois', 'Trignac', '14e arrondissement', 'Gradignan',
       'Leymen', 'Evry', 'Saint-Raphaël', 'Mandelieu-La-Napoule',
       'Mantes-La-Jolie ', 'Commelle-Vernay', 'Marseille', 'Morre',
       'Wittenheim', 'Bora Bora', '

In [108]:
df_all['commune_sans_accent'] = (df_all['commune']
                                 .str.lower()
                                 .str.normalize('NFKD')
                                 .str.encode('ascii', errors='ignore')
                                 .str.decode('utf-8')
)
df_all['departement_sans_accents'] = (df_all['departement']
                                       .str.lower()
                                       .str.normalize('NFKD')
                                       .str.encode('ascii', errors='ignore')
                                       .str.decode('utf-8')
)
df_all.head()

,date,commune,departement,prenom,age,commune_sans_accent,departement_sans_accents
0,25/11/2022,Prahecq,Deux-Sèvres,Leslie,22.0,prahecq,deux-sevres
1,04/01/2023,Perles,Aisne,–,50.0,perles,aisne
2,31/12/2022,Charleville-Mézières,Ardennes,–,23.0,charleville-mezieres,ardennes
3,24/12/2022,Saint-Raphaël,Var,Molka,31.0,saint-raphael,var
4,16/10/2022,Goudelin,Côtes-d'Armor,Jacqueline,67.0,goudelin,cotes-d'armor


In [109]:
#date en format datetime
df_all['date'] = pd.to_datetime(df_all['date'], format='%d/%m/%Y', errors='coerce')
df_all.head()

,date,commune,departement,prenom,age,commune_sans_accent,departement_sans_accents
0,2022-11-25,Prahecq,Deux-Sèvres,Leslie,22.0,prahecq,deux-sevres
1,2023-01-04,Perles,Aisne,–,50.0,perles,aisne
2,2022-12-31,Charleville-Mézières,Ardennes,–,23.0,charleville-mezieres,ardennes
3,2022-12-24,Saint-Raphaël,Var,Molka,31.0,saint-raphael,var
4,2022-10-16,Goudelin,Côtes-d'Armor,Jacqueline,67.0,goudelin,cotes-d'armor


In [115]:
#dans df_2022, remplacer les noms de commune contenant '*e arrondissement' par 'Paris'
df_all['commune_sans_accent'] = df_all['commune_sans_accent'].str.replace(r'\d{1,2}e arrondissement', 'paris', regex=True)
df_all[df_all['commune_sans_accent'].str.contains('lyon')]

,date,commune,departement,prenom,age,commune_sans_accent,departement_sans_accents
99,2022-05-08,Bourgoin-Jallieu / Lyon,Rhône,Julie,26.0,bourgoin-jallieu / lyon,rhone
155,2023-08-09,Lyon,Rhône,Zohra,38.0,lyon,rhone
222,2023-06-09,Lyon,Rhône,Alicia,18.0,lyon,rhone
403,2024-02-11,Sainte-Foy-lès-Lyon,Rhône,Cynthia,45.0,sainte-foy-les-lyon,rhone


In [ ]:

df_all['commune_sans_accent'] = df_all['commune_sans_accent'].str.replace('bourgoin-jallieu / lyon', 'bourgoin-jallieu')


,date,commune,departement,prenom,age,commune_sans_accent,departement_sans_accents
155,2023-08-09,Lyon,Rhône,Zohra,38.0,lyon,rhone
222,2023-06-09,Lyon,Rhône,Alicia,18.0,lyon,rhone
403,2024-02-11,Sainte-Foy-lès-Lyon,Rhône,Cynthia,45.0,sainte-foy-les-lyon,rhone


In [ ]:
df_all.loc[df_all["departement_sans_accents"].str.contains("hors france"), "commune_sans_accent"] = "hors france"
df_all.loc[df_all["departement_sans_accents"].str.contains("hors france"), "departement_sans_accent"] = "hors france"
df_all['departement_sans_accents'].unique()

array(['deux-sevres', 'aisne', 'ardennes', 'var', "cotes-d'armor",
       'guyane', 'paris', 'haut-rhin', 'nord', 'seine-saint-denis',
       'meurthe-et-moselle', 'la reunion', 'gironde', 'rhone', 'dordogne',
       'orne', 'ain', 'seine-maritime', 'finistere', 'mayenne', 'essonne',
       'marne', 'ille-et-vilaine', 'eure', 'lot-et-garonne', 'vienne',
       'gard', "cote-d'or", 'bouches-du-rhone', 'martinique',
       'hauts-de-seine', 'pyrenees-orientales', 'correze',
       'loire-atlantique', 'corse-du-sud', 'polynesie francaise',
       'loir-et-cher', 'alpes-maritimes', 'yvelines', 'loire', 'doubs',
       'jura', 'loiret', 'vaucluse', 'eure-et-loir', 'hautes-alpes',
       'nouvelle-caledonie', 'hautes-pyrenees', 'morbihan', "val-d'oise",
       'sarthe', 'aude', 'saone-et-loire', 'indre-et-loire', 'herault',
       'haute-garonne', 'vosges', 'somme', 'val-de-marne', 'guadeloupe',
       'moselle', 'haute-corse', 'isere', 'maine-et-loire', 'bas-rhin',
       'seine-et-marne', 

In [131]:
df_all.drop(['departement_sans_accent'], axis=1, inplace=True)
df_all.rename(columns={'departement_sans_accents': 'departement_sans_accent'}, inplace=True)

In [136]:
df_all['commune_sans_accent'].unique()

array(['prahecq', 'perles', 'charleville-mezieres', 'saint-raphael ',
       'goudelin', 'soula', 'paris', 'mulhouse', 'lannoy', 'pantin',
       'mont-saint-martin', 'tampon', 'floirac', 'grand-fayt',
       'villeurbanne', "saint-pierre-d'eyraud", 'valframbert',
       'saint-jean-de-gonville', 'rouen', 'quimper', 'brece',
       'evry-courcouronnes', 'pargny-sur-saulx', 'pavillons-sous-bois ',
       'saint-jacques-de-la-lande', 'saint-jacques-de-la-lande ',
       'louviers ', 'tonneins', 'savigne', 'montaren-et-saint-mediers',
       'diancey', "berre-l'etang", 'ronchin', 'ducos', 'sainte-rose',
       'stains', 'neuvy ', 'carantec', 'meudon', 'ille-sur-tet',
       'brive la gaillarde', 'nantes', 'ajaccio', 'taravao', 'blois',
       'trignac', 'gradignan', 'leymen', 'evry', 'saint-raphael',
       'mandelieu-la-napoule', 'mantes-la-jolie ', 'commelle-vernay',
       'marseille', 'morre', 'wittenheim', 'bora bora',
       'boulazac-isle-manoire', 'alieze', 'carrieres-sur-seine',


In [137]:
df_all.loc[df_all['commune_sans_accent']=="mamoudzou (m'tsapere)", 'commune_sans_accent'] = "mamoudzou"
df_all.loc[df_all['commune_sans_accent']=='corne (loire-authion)', 'commune_sans_accent'] = "corne"
df_all.loc[df_all['commune_sans_accent']=='paris (18e)', 'commune_sans_accent'] = "paris"
df_all.loc[df_all['commune_sans_accent']=='saint pierre (ravine des cabris)', 'commune_sans_accent'] = "saint-pierre"



In [141]:
df_all['commune_sans_accent'].str.startswith("l'").unique()

array([False])

In [222]:
df_commune = pd.read_csv('../data/raw/communes-france-2025.csv')
df_commune.head()

C:\Users\ghirg\AppData\Local\Temp\ipykernel_17008\954040633.py:1: DtypeWarning: Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_commune = pd.read_csv('../data/raw/communes-france-2025.csv')


,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
0,0,01001,L'Abergement-Clémenciat,Abergement-Clémenciat,à Abergement-Clémenciat,de l'Abergement-Clémenciat,l-abergement-clemenciat,L'ABERGEMENT-CLÉMENCIAT,COM,commune,...,4.921,46.153,4.926,6,Rural à habitat dispersé,0.0,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01001-l-abergemen...
1,1,01002,L'Abergement-de-Varey,Abergement-de-Varey,à Abergement-de-Varey,de l'Abergement-de-Varey,l-abergement-de-varey,L'ABERGEMENT-DE-VAREY,COM,commune,...,5.423,46.009,5.428,6,Rural à habitat dispersé,0.0,communes non pôle,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01002-l-abergemen...
2,2,01004,Ambérieu-en-Bugey,Ambérieu-en-Bugey,à Ambérieu-en-Bugey,d'Ambérieu-en-Bugey,amberieu-en-bugey,AMBÉRIEU-EN-BUGEY,COM,commune,...,5.360,45.961,5.373,2,Centres urbains intermédiaires,3.0,centres structurants d'équipements et de services,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...,https://villedereve.fr/ville/01004-amberieu-en...
3,3,01005,Ambérieux-en-Dombes,Ambérieux-en-Dombes,à Ambérieux-en-Dombes,d'Ambérieux-en-Dombes,amberieux-en-dombes,AMBÉRIEUX-EN-DOMBES,COM,commune,...,4.903,45.996,4.912,5,Bourgs ruraux,1.0,centres locaux d'équipements et de services,Ambarrois,https://fr.wikipedia.org/wiki/fr:Ambérieux-en-...,https://villedereve.fr/ville/01005-amberieux-e...
4,4,01006,Ambléon,Ambléon,à Ambléon,d'Ambléon,ambleon,AMBLÉON,COM,commune,...,5.601,45.750,5.594,6,Rural à habitat dispersé,0.0,communes non pôle,Ambléonais,https://fr.wikipedia.org/wiki/fr:Ambléon,https://villedereve.fr/ville/01006-ambleon


In [223]:
# dans df_commune, récupérer commmune contenant 'Paris'
df_commune = df_commune[['nom_sans_accent','dep_nom','dep_code','population','superficie_km2','densite','latitude_mairie','longitude_mairie']]

In [224]:
df_commune.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34935 entries, 0 to 34934
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nom_sans_accent   34935 non-null  object 
 1   dep_nom           34935 non-null  object 
 2   dep_code          34935 non-null  object 
 3   population        34935 non-null  int64  
 4   superficie_km2    34935 non-null  int64  
 5   densite           34932 non-null  float64
 6   latitude_mairie   34935 non-null  float64
 7   longitude_mairie  34935 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 2.1+ MB


In [225]:
df_commune['nom_sans_accent'] = df_commune['nom_sans_accent'].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_commune['dep_nom'] = df_commune['dep_nom'].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [226]:
df_commune.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34935 entries, 0 to 34934
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nom_sans_accent   34935 non-null  object 
 1   dep_nom           34935 non-null  object 
 2   dep_code          34935 non-null  object 
 3   population        34935 non-null  int64  
 4   superficie_km2    34935 non-null  int64  
 5   densite           34932 non-null  float64
 6   latitude_mairie   34935 non-null  float64
 7   longitude_mairie  34935 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 2.1+ MB


In [227]:
#inner join entre df_2022 et df_commune sur les colonnes 'commune_sans_accents' et 'departement_sans_accents' de df_2022 et 'nom_sans_accent' et 'dep_sans_accents' de df_commune
df_merged = pd.merge(df_all, 
                     df_commune, 
                     how='left', 
                     left_on=['commune_sans_accent','departement_sans_accent'], 
                     right_on=['nom_sans_accent','dep_nom'])

In [233]:
 #enlever les lignes où nom_sans_accent est NaN
df_merged = df_merged[~df_merged['nom_sans_accent'].isna()]
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 390 entries, 0 to 527
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     390 non-null    datetime64[ns]
 1   commune                  390 non-null    object        
 2   departement              390 non-null    object        
 3   prenom                   389 non-null    object        
 4   age                      389 non-null    float64       
 5   commune_sans_accent      390 non-null    object        
 6   departement_sans_accent  390 non-null    object        
 7   nom_sans_accent          390 non-null    object        
 8   dep_nom                  390 non-null    object        
 9   dep_code                 390 non-null    object        
 10  population               390 non-null    float64       
 11  superficie_km2           390 non-null    float64       
 12  densite                  390 non-null    

In [236]:
df_merged.rename(columns={'latitude_mairie': 'latitude', 'longitude_mairie': 'longitude'}, inplace=True)

In [248]:
df_merged['nb_par_departement'] = df_merged.groupby('departement_sans_accent')['commune_sans_accent'].transform('count')
df_merged['frequence_par_habitant'] = df_merged['nb_par_departement'] / df_merged['population']
#convertir age en int sans les nan
df_merged['age'] = df_merged['age'].astype('Int64')

In [249]:
df_merged.head()

,date,commune,departement,prenom,age,commune_sans_accent,departement_sans_accent,nom_sans_accent,dep_nom,dep_code,population,superficie_km2,densite,latitude,longitude,nb_par_departement,frequence_par_habitant
0,2022-11-25,Prahecq,Deux-Sèvres,Leslie,22,prahecq,deux-sevres,prahecq,deux-sevres,79,2246.0,25.0,90.0,46.259,-0.345,2,0.000890
1,2023-01-04,Perles,Aisne,–,50,les-septvallons,aisne,les-septvallons,aisne,02,1215.0,9.0,142.0,49.359,3.654,4,0.003292
2,2022-12-31,Charleville-Mézières,Ardennes,–,23,charleville-mezieres,ardennes,charleville-mezieres,ardennes,08,46398.0,32.0,1460.0,49.760,4.720,3,0.000065
4,2022-10-16,Goudelin,Côtes-d'Armor,Jacqueline,67,goudelin,cotes-d'armor,goudelin,cotes-d'armor,22,1720.0,23.0,74.0,48.605,-3.016,4,0.002326
6,2022-07-28,13e arrondissement,Paris,Florence,48,paris,paris,paris,paris,75,2133111.0,105.0,20238.0,48.857,2.352,22,0.000010


In [250]:
df_merged.to_csv('../data/processed/feminicide_2022_2025.csv', index=False)

In [231]:
df_merge_nan = df_merged[df_merged['nom_sans_accent'].isna()]
df_merge_nan

,date,commune,departement,prenom,age,commune_sans_accent,departement_sans_accent,nom_sans_accent,dep_nom,dep_code,population,superficie_km2,densite,latitude_mairie,longitude_mairie
3,2022-12-24,Saint-Raphaël,Var,Molka,31.0,saint-raphael,var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2022-12-30,Soula,Guyane,–,30.0,soula,guyane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2022-09-03,Tampon,La Réunion,Dominique,68.0,tampon,la reunion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2022-12-25,Saint-Pierre-d'Eyraud,Dordogne,Dominique,64.0,saint-pierre-d'eyraud,dordogne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2022-12-07,Pavillons-sous-Bois,Seine-Saint-Denis,–,51.0,pavillons-sous-bois,seine-saint-denis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,2025-02-06,Villefranche de rouergue,Aveyron,Dawn,63.0,villefranche de rouergue,aveyron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,2025-02-11,Offenbourg,Autre ou hors France,Elena,37.0,hors france,hors france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525,2025-01-07,Brétigny sur orge,Essonne,Sandy,34.0,bretigny sur orge,essonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
526,2025-01-04,Yaté,Nouvelle-Calédonie,Graziella,54.0,yate,nouvelle-caledonie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [211]:
df_commune[df_commune['nom_sans_accent'].str.contains('saint-raphael')]

,nom_sans_accent,dep_nom,dep_code,population,superficie_km2,densite,latitude_mairie,longitude_mairie
8291,saint-raphael,dordogne,24,93,7,13.0,45.304,1.074
32372,saint-raphael,var,83,35950,91,395.0,43.425,6.768


In [210]:
df_all[df_all['commune_sans_accent']=='saint-raphael']

,date,commune,departement,prenom,age,commune_sans_accent,departement_sans_accent
51,2022-09-22,Saint-Raphaël,Var,Edith,60.0,saint-raphael,var
407,2024-02-04,Saint-Raphaël,Var,–,77.0,saint-raphael,var


In [209]:
df_all.loc[(df_all['commune_sans_accent']=='perles') & (df_all["departement_sans_accent"] == 'aisne'),'commune_sans_accent'] = "les-septvallons"

In [214]:
df_all['departement_sans_accent'].unique()

array(['deux-sevres', 'aisne', 'ardennes', 'var', "cotes-d'armor",
       'guyane', 'paris', 'haut-rhin', 'nord', 'seine-saint-denis',
       'meurthe-et-moselle', 'la reunion', 'gironde', 'rhone', 'dordogne',
       'orne', 'ain', 'seine-maritime', 'finistere', 'mayenne', 'essonne',
       'marne', 'ille-et-vilaine', 'eure', 'lot-et-garonne', 'vienne',
       'gard', "cote-d'or", 'bouches-du-rhone', 'martinique',
       'hauts-de-seine', 'pyrenees-orientales', 'correze',
       'loire-atlantique', 'corse-du-sud', 'polynesie francaise',
       'loir-et-cher', 'alpes-maritimes', 'yvelines', 'loire', 'doubs',
       'jura', 'loiret', 'vaucluse', 'eure-et-loir', 'hautes-alpes',
       'nouvelle-caledonie', 'hautes-pyrenees', 'morbihan', "val-d'oise",
       'sarthe', 'aude', 'saone-et-loire', 'indre-et-loire', 'herault',
       'haute-garonne', 'vosges', 'somme', 'val-de-marne', 'guadeloupe',
       'moselle', 'haute-corse', 'isere', 'maine-et-loire', 'bas-rhin',
       'seine-et-marne', 

In [218]:
df_commune['dep_nom'].unique()

array(['ain', 'aisne', 'allier', 'alpes-de-haute-provence',
       'hautes-alpes', 'alpes-maritimes', 'ardeche', 'ardennes', 'ariege',
       'aube', 'aude', 'aveyron', 'bouches-du-rhone', 'calvados',
       'cantal', 'charente', 'charente-maritime', 'cher', 'correze',
       "cote-d'or", "cotes-d'armor", 'creuse', 'dordogne', 'doubs',
       'drome', 'eure', 'eure-et-loir', 'finistere', 'corse-du-sud',
       'haute-corse', 'gard', 'haute-garonne', 'gers', 'gironde',
       'herault', 'ille-et-vilaine', 'indre', 'indre-et-loire', 'isere',
       'jura', 'landes', 'loir-et-cher', 'loire', 'haute-loire',
       'loire-atlantique', 'loiret', 'lot', 'lot-et-garonne', 'lozere',
       'maine-et-loire', 'manche', 'marne', 'haute-marne', 'mayenne',
       'meurthe-et-moselle', 'meuse', 'morbihan', 'moselle', 'nievre',
       'nord', 'oise', 'orne', 'pas-de-calais', 'puy-de-dome',
       'pyrenees-atlantiques', 'hautes-pyrenees', 'pyrenees-orientales',
       'bas-rhin', 'haut-rhin', 'rhone',

In [220]:
set(df_all.loc[df_all['commune_sans_accent']=="saint-raphael", 'departement_sans_accent'])


{'var'}

In [221]:
set(df_commune.loc[df_commune['nom_sans_accent']=="saint-raphael", 'dep_nom'])

{'dordogne', 'var'}